## Measuring Bell magic

Python code to measure Bell magic using two copies of a state in the Bell basis. Simulates various states with optional depolarizing noise and finite number of measurements. Runs on qiskit.

Companion Code for "Scalable measures of magic for quantum computers" by T. Haug, M.S. Kim

@Tobias Haug, Imperial College London
thaug@ic.ac.uk

In [1]:
import qiskit as qk
import numpy as np

In [13]:
def numberToBase(n, b,n_qubits):
    ##get all possible bases
    if n == 0:
        return np.zeros(n_qubits,dtype=int)
    digits = np.zeros(n_qubits,dtype=int)
    counter=0
    while n:
        digits[counter]=int(n % b)
        n //= b
        counter+=1
    return digits[::-1]


def get_pauli_commute_map():
    #maps whether two pauli single qubit commute onto a single array
    #This helps to vectorize and speed up calculation of bell magic

    x1=np.arange(4)
    x2=np.arange(4)
    map_pauli_commute=np.zeros(16)
    for i in range(4):
        for j in range(4):
            map_pauli_commute[4*i+j]=does_pauli_not_commute(x1[i],x2[j])
    
    return map_pauli_commute

def does_pauli_not_commute(pauli1,pauli2):
    #determines whether two single-qubit pauli operators commute (0) or not commute (1)

    if(pauli1!=pauli2 and pauli1!=0 and pauli2!=0):
        return 1
    else:
        return 0

def bell_magic_exact(bitstrings_sampled,sampled_states_probs):
    """
    get exact Bell magic for infinite number of samples. 
    Only correct result if number of samples is infinite, else this is a biased estimator.
    bitstrings_sampled: bitstrings from Bell measurement, first copy on first n_qubit bits, second copy on last n_qubit bits
    sampled_states_probs: exact probabilities of sampling bitstring
    """


    ##maps each single-qubit pauli operator pair to whether it commutes or not, input is 4*pauli1+pauli2
    pauli_commute_map=get_pauli_commute_map()

    
    string_length=len(bitstrings_sampled[0])
    n_qubits=string_length//2
    ##inputs=[]

    n_bitstring_samples=len(bitstrings_sampled)
    bitstrings_added_list=np.zeros([((n_bitstring_samples-1)*n_bitstring_samples)//2,string_length],dtype=int)
    bitstrings_added_probs=np.zeros([((n_bitstring_samples-1)*n_bitstring_samples)//2])

    start=0
    ##go over upper triangular elements
    for q in range(n_bitstring_samples-1):
        bitstrings_added_list[start:start+n_bitstring_samples-q-1,:]=(bitstrings_sampled[q]+bitstrings_sampled[q+1:])%2
        bitstrings_added_probs[start:start+n_bitstring_samples-q-1]=2*sampled_states_probs[q]*sampled_states_probs[q+1:]
        start+= n_bitstring_samples-q-1
        
        

    ##remove doubled bitstrings and add the probabililies together
    bitstrings_added_probs=np.array(bitstrings_added_probs)
    bitstrings_added_list_unique,bitstrings_added_list_unique_inverse=np.unique(bitstrings_added_list,axis=0,return_inverse=True)
    bitstrings_added_probs_unique=np.zeros(len(bitstrings_added_list_unique))
    for i in range(len(bitstrings_added_list_unique_inverse)):
        bitstrings_added_probs_unique[bitstrings_added_list_unique_inverse[i]]+=bitstrings_added_probs[i]

  
    
    ##get basis in Pauli basis instead of bitbasis
    sampled_basis4=[2*bitstrings_added_list_unique[k][:n_qubits]+bitstrings_added_list_unique[k][n_qubits:] for k in range(len(bitstrings_added_list_unique))]
        
    


    ##get non-commuting bitstrings, vectorized operation
    not_commute_probs_sampled=np.zeros([len(bitstrings_added_list_unique),len(bitstrings_added_list_unique)])  
    for q in range(len(bitstrings_added_list_unique)):
        not_commute_probs_sampled[q,:]=bitstrings_added_probs_unique[q]*bitstrings_added_probs_unique*(np.sum(pauli_commute_map[4*sampled_basis4[q]+sampled_basis4],axis=1)%2)


    Bell_magic_exact=np.sum(2*not_commute_probs_sampled) ##add factor of 2 to account for infinity norm of commutator

    return Bell_magic_exact

def SWAP_purity(bitstrings_sampled,sampled_states_probs):
    """
    get purity from bitstrings and probabilities of Bell measurement
    """
    string_length=len(bitstrings_sampled[0])
    n_qubits=string_length//2

    ##do AND between each pair of bits from first and second copy, then check parity
    parity_list=np.array([np.sum((bitstrings_sampled[i,:n_qubits]+bitstrings_sampled[i,n_qubits:])>1)%2 for i in range(len(bitstrings_sampled))],dtype=int)

    ##purity is 1-2*(probability of odd parity)
    purity=1-2*np.sum(sampled_states_probs*parity_list)
    return purity

def bell_magic_sample(bitstrings_sampled,sampled_states_probs, n_samples=0, n_resample=1):
    """
    Estimate Bell magic from samples of Bell measurements
    Inputs are 
    bitstrings_sampled: sampled bitstrings from Bell measurement, first copy on first n_qubit bits, second copy on last n_qubit bits
    sampled_states_probs: respective probabilities
    n_samples: total number of measurements
    n_resample: How often to repeat magic estimation protocol, good choice is 10*n_samples
    
    """

    ##maps each single-qubit pauli operator pair to whether it commutes or not, input is 4*pauli1+pauli2
    pauli_commute_map=get_pauli_commute_map()

    string_length=len(bitstrings_sampled[0])
    n_qubits=string_length//2

    if(n_samples==0):
        raise NameError("Not implemented")
    
    ##reconstruct counts to verifify that n_samples is indeed correct
    counts=np.array(np.round(sampled_states_probs*n_samples),dtype=int)
    if(np.sum(counts)!=n_samples):
        raise NameError("n_samples did not match acutal counts")
    
    ##list that numbers each sample individually
    sample_list=np.zeros(n_samples,dtype=int) ##list numbering each outcome with corresponding bitstring
    start=0
    for i in range(len(bitstrings_sampled)):
        sample_list[start:start+counts[i]]=i
        start+=counts[i]
        

    ##whether to sample with or without replacement
    if(n_samples<4):
        replace=True
    else:
        replace=False
    
    ##sample 4 random bitstrings from all samples, do this n_resample times
    rep_sample_list=[]
    for rep in range(n_resample):
        rep_sample_list.append(rng.choice(sample_list,4,replace=replace))
    
    rep_sample_list=np.array(rep_sample_list)

    ##add bitstrings together
    bitstring_added1=(bitstrings_sampled[rep_sample_list[:,0]]+bitstrings_sampled[rep_sample_list[:,1]])%2
    bitstring_added2=(bitstrings_sampled[rep_sample_list[:,2]]+bitstrings_sampled[rep_sample_list[:,3]])%2
           
    ##get basis in Pauli basis instead of bitbasis
    sampled_basis4_1=2*bitstring_added1[:,:n_qubits]+bitstring_added1[:,n_qubits:]
    sampled_basis4_2=2*bitstring_added2[:,:n_qubits]+bitstring_added2[:,n_qubits:]   
    
    
    ##get non-commuting paulis
    sum_noncommute_return_list=np.mean(2*(np.sum(pauli_commute_map[4*sampled_basis4_1+sampled_basis4_2],axis=1)%2))
    
    ##magic is probability of sampling non-commuting bitstrings
    Bell_magic_sampled=np.mean(sum_noncommute_return_list)

    return Bell_magic_sampled

In [3]:
def get_qiskit_circuit_tensor(circuit_parameters,rotation_gates,entangling_gate_index_list,get_statevector=False):
    """
    creates qiskit circuit for measuring Bell magic. 
    Prepares two copies of state of various types, then Bell transformation, then sampling in computational basis
    """
    depth=np.shape(circuit_parameters)[0]
    n_qubits=np.shape(circuit_parameters)[1]
    n_qubits_tensor=2*n_qubits
    qreg = qk.QuantumRegister(n_qubits_tensor)

    if(get_statevector==False):
        creg = qk.ClassicalRegister(n_qubits_tensor)
        circuit = qk.QuantumCircuit(qreg,creg)
    else:
        circuit = qk.QuantumCircuit(qreg)
        

    ##go through all layers of parameterized quantum circuit
    for j in range(depth):
        ##add parameterized rotation for first copy
        for k in range(n_qubits):
            angle=circuit_parameters[j][k]
            type_pauli=rotation_gates[j][k]
            if(type_pauli==1):
                circuit.rx(angle,k)
            elif(type_pauli==2):
                circuit.ry(angle,k)
            elif(type_pauli==3):
                circuit.rz(angle,k)
                
        ##add parameterized rotation for second copy
        for k in range(n_qubits):
            angle=circuit_parameters[j][k]
            type_pauli=rotation_gates[j][k]
            if(type_pauli==1):
                circuit.rx(angle,k+n_qubits)
            elif(type_pauli==2):
                circuit.ry(angle,k+n_qubits)
            elif(type_pauli==3):
                circuit.rz(angle,k+n_qubits)
    
        
        if(len(entangling_gate_index_list[j])>0):
            ##add entangling cnot gates for first copy
            for gate_indices in entangling_gate_index_list[j]:
                circuit.cnot(gate_indices[0],gate_indices[1])
                
            ##add entangling cnot gates for second copy
            for gate_indices in entangling_gate_index_list[j]:
                circuit.cnot(gate_indices[0]+n_qubits,gate_indices[1]+n_qubits)
        
    ##bell measurement
    for k in range(n_qubits):
        circuit.cnot(k,k+n_qubits)
        circuit.h(k)
    
    ##add measurements
    if(get_statevector==False):
        circuit.barrier()
        circuit.measure(qreg,creg)
        
        
    return circuit

In [62]:
##parameters to be adjusted

n_qubits=3 #number qubits
depth=4 #number of layers of parameterized single qubit rotations, (twice of d as defined in manuscript)

type_circuit=2##0: A-type magic-state, 1: T-type magic state, 2: circuit with y,z rotations and CNOT in nearest neighbor chain configuration, 3: + state

type_parameters=2 ##0: random parameters, 1: random clifford parameters \in\{0,\pi/2,\pi,3\pi/2} + n_nonclifford parameters which represent T-gates, 2: random clifford parameter with initial layer of n_nonclifford magic states
n_nonclifford=1 #number of equivalent T-gates for type_parameters=1 or number of magic states for type_parameters=2

depolarization_factor=0.1 ##depolarization noise of acting individually on each copy of the two states


random_seed=2 #seed of random generator

n_samples=0 ##number of measurements, for 0: do statevector simulation with infinite number of measurements
n_resample=10*n_samples ##number of resampling steps in postprocessing for Bell magic

In [63]:
#random generator used
rng = np.random.default_rng(random_seed)


In [64]:
#define parameters for circuit, not used for t-
circuit_parameters=np.zeros([depth,n_qubits])
if(type_parameters==0): ##random angles
    circuit_parameters=rng.random([depth,n_qubits])*2*np.pi
elif(type_parameters==1): ##random clifford angles with n_nonclifford non-clifford parameters introducing T-gates
    circuit_parameters=rng.integers(0,4,[depth,n_qubits])*np.pi/2
    which_params=np.arange(depth*n_qubits)
    rng.shuffle(which_params)
    for i in range(n_nonclifford):
        circuit_parameters[which_params[i]//n_qubits,which_params[i]%n_qubits]=rng.integers(0,4)*np.pi/2+np.pi/4
elif(type_parameters==2): ##n_nonclifford initial magic states and random clifford angles 
    circuit_parameters=rng.integers(0,4,[depth,n_qubits])*np.pi/2
    which_params=np.arange(n_qubits)
    rng.shuffle(which_params)
    #transform initial states (i.e. first layer) into n_nonclifford magic states on random qubits
    for i in range(n_nonclifford):
        circuit_parameters[0,which_params[i]]+=np.pi/4 ##transform into magic state



In [65]:
##define here various types of circuits
if(type_circuit==0): ##A type magic state
    rotation_gates=np.zeros([depth,n_qubits])
    rotation_gates[0,:]=2
    rotation_gates[1,:]=3
    
    circuit_parameters[0,:]=np.pi/2
    circuit_parameters[1,:]=np.pi/4
    
    entangling_gate_index_list=[[] for j in range(depth)]
elif(type_circuit==1): ##T-type magic state
    rotation_gates=np.zeros([depth,n_qubits])
    rotation_gates[0,:]=2
    rotation_gates[1,:]=3
    
    circuit_parameters[0,:]=np.arccos(1/np.sqrt(3))
    circuit_parameters[1,:]=np.pi/4
    
    entangling_gate_index_list=[[] for j in range(depth)]

elif(type_circuit==2): ##yz circuit with CNOT in nearest-neighbor chain configuration
    rotation_gates=np.zeros([depth,n_qubits])
    entangling_gate_index_list=[]
    for j in range(depth):
        if(j%2==0):
            rotation_gates[j,:]=2
            entangling_gate_index_list.append([])
        else:
            rotation_gates[j,:]=3
            if(j<depth-1):
                if(n_qubits==2):
                    entangling_gate_index_list.append([[0,1]])
                else:
                    entangling_gate_index_list.append([[2*j,(2*j+1)] for j in range(n_qubits//2)]+[[2*j+1,(2*j+2)] for j in range((n_qubits-1)//2)])
            else:
                entangling_gate_index_list.append([])

elif(type_circuit==3): ##+ state
    rotation_gates=np.zeros([depth,n_qubits])
    rotation_gates[0,:]=2
    
    circuit_parameters[0,:]=np.pi/2

    entangling_gate_index_list=[[] for j in range(depth)]
    

In [66]:
##load IBMQ
qk.IBMQ.load_account()
qk.IBMQ.providers()
provider = qk.IBMQ.get_provider(hub='ibm-q', group='open', project='main')
if(n_samples==0):
    backend = qk.Aer.get_backend('statevector_simulator')
else:
    backend = qk.Aer.get_backend('qasm_simulator')



ibmqfactory.load_account:WARNING:2022-04-04 22:37:52,809: Credentials are already in use. The existing account in the session will be replaced.


In [67]:
##prepare circuit, different details depending whether we use statevector simulator or qasm
if(n_samples==0):
    qiskit_circuit=get_qiskit_circuit_tensor(circuit_parameters,rotation_gates,entangling_gate_index_list,get_statevector=True)
else:
    qiskit_circuit=get_qiskit_circuit_tensor(circuit_parameters,rotation_gates,entangling_gate_index_list)

In [68]:
##run job
if(n_samples==0):
    job_qiskit = qk.execute(qiskit_circuit,backend=backend)
else:
    job_qiskit = qk.execute(qiskit_circuit,backend=backend,shots=n_samples)

In [69]:
##get result from IBMQ
job_qiskit_result=job_qiskit.result()

In [70]:
##evaluate counts by turning them into a probability vector
##note that we need to transform from little to big endian for the probability vector

global_depolarization=depolarization_factor*(2-depolarization_factor) #relates depolarization on invidividual copies to global depolarization over two copies. For Bell measurement, this is exact
if(n_samples==0):#for statevector simulator get probabilities and all possible bitstrings
    sampled_bitstrings_probability=np.abs(job_qiskit_result.get_statevector())**2
    base_states_qiskit_order=[numberToBase(i, 2,2*n_qubits)[::-1] for i in range(4**n_qubits)] #base states as written by qiskit. number of state is ordered according to big endian
    sampled_bitstrings=base_states_qiskit_order
    if(global_depolarization>0):
        ##global depolarization leads to any possible bitstring appear with probabilitiy global_depolarization/4**n_qubits
        sampled_bitstrings_probability=(1-global_depolarization)*sampled_bitstrings_probability+global_depolarization/4**n_qubits*np.ones(4**n_qubits)
    
else: #get estimate of sampling probabilities from measured counts
    dict_counts=job_qiskit_result.get_counts(0)
    sampled_bitstrings=[]
    sampled_bitstrings_probability=[]
    
    #go through all sampled bitstrings
    for key in list(dict_counts.keys()):
        bitstring=[int(key[i]) for i in range(len(key))]
        counts=dict_counts[key]
        #go through all counts and add each bitstring individually correspondong probability 1/n_samples
        for i in range(counts):
            prob=1/n_samples
            sampled_bitstrings_probability.append(prob)
            if(global_depolarization>0):
                rng_depol=rng.random()
                if(rng_depol<depolarization_factor):#with chance of global_depolarization, add random bitstring instead of actually measured one
                    random_bitstring=rng.integers(0,2,2*n_qubits)
                    sampled_bitstrings.append(random_bitstring)
                else:
                    sampled_bitstrings.append(bitstring)
            else:##add bitstrings to sampled bitstrings
                sampled_bitstrings.append(bitstring)

        
        

sampled_bitstrings=np.array(sampled_bitstrings)
sampled_bitstrings_probability=np.array(sampled_bitstrings_probability)

##calculate magic
if(n_samples==0):
    bell_magic=bell_magic_exact(sampled_bitstrings,sampled_bitstrings_probability)
else:
    bell_magic=bell_magic_sample(sampled_bitstrings,sampled_bitstrings_probability,n_samples=n_samples,n_resample=n_resample)
    
bell_magic_add=-np.log2(1-bell_magic)
print("Bell Magic",bell_magic)
print("Additive Bell magic",bell_magic_add)


##calculate purity
purity=SWAP_purity(sampled_bitstrings,sampled_bitstrings_probability)
print("Purity",purity)


Bell Magic 0.50274
Additive Bell magic 1.0079277106451485
Purity 1.0


In [71]:
#mitigate magic

hilbertspace=2**n_qubits
prob_Identity_addedstring=np.sum(sampled_bitstrings_probability**2) #probability of same bitstrings appearing, lower bounded by 4**-n_qubits and upper bounded by 2**-n_qubits


depol_probability=1-np.sqrt((hilbertspace-1)*(hilbertspace*purity-1))/(hilbertspace-1) ##get depolariziation probability from purity
depol_4=1-(1-depol_probability)**4 
zero_string_prob_mtg=(prob_Identity_addedstring-4**(-n_qubits)*depol_4)/(1-depol_4)
zero_string_prob_mtg=min(max(zero_string_prob_mtg,4**(-n_qubits)),2**(-n_qubits))
B_R=1-zero_string_prob_mtg##alternativly, can be simply approximated by 1-4**(-n_qubits) 
B_totallymixed=1-4**(-n_qubits) #magic of totally mixed state
bell_magic_mtg=1/(1-depol_4)**2*(bell_magic-depol_4**2*B_totallymixed-2*depol_4*(1-depol_4)*B_R)

bell_magic_add_mtg=-np.log2(1-bell_magic_mtg)
print("Mitigated Bell magic",bell_magic_mtg)
print("Mitigated additive Bell magic",bell_magic_add_mtg)

Mitigated Bell magic 0.50274
Mitigated additive Bell magic 1.0079277106451485
